<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-02 13:18:26
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.24 L
Current:  1.33 C
-------------------
Today PnL: 45.88 K (0.35%)
Current PnL: -15.90 L (-11.08%)
CY Booked + Current PnL: -11.43 L (-7.96%)
-------------------
Total profit:  4.34 L
Total loss:  -20.24 L
-------------------
Total Booked + Current PnL: 15.55 L (13.35%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.38%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 79.02 L (59.63%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.40 C (105.89%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LICI,785.05,983.0,-2.96,H-LC,91.40,238762.0,44070.0,5014.0,0.87,22.64,2.10,25.21,29.0,8.79,1.86,32.03,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,82.80,104768.0,26406.0,7627.0,2.99,33.70,7.28,43.43,71.0,3.46,0.82,40.13,XY24,NTT,MISC
85,UNIONBANK,123.87,163.0,-0.31,M-LC,93.55,174859.0,34019.0,10474.0,4.77,24.15,5.99,31.59,76.0,3.25,1.36,51.89,XY24,NTT,BANKS
68,SBIN,760.30,863.0,-11.57,M-LC,46.24,216770.0,15290.0,11922.0,0.70,7.59,5.50,13.51,73.0,1.28,1.69,21.14,XY25,NTT,BANKS
62,RELIANCE,1291.83,1526.0,3.85,X-LC,17.20,154126.0,13317.0,12207.0,-0.49,9.46,7.92,18.13,17.0,1.09,1.20,21.70,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UNIONBANK,123.87,163.00,-0.31,M-LC,93.55,174859.0,34019.0,10474.0,4.77,24.15,5.99,31.59,76.0,3.25,1.36,51.89,XY24,NTT,BANKS
17,CAMS,3643.00,5250.99,3.33,H-SC,51.61,116144.0,14140.0,30883.0,4.72,13.86,26.59,44.14,111.0,0.46,0.91,32.35,XR,ATH,MISC
75,SURYODAY,189.78,240.00,62.28,H-SC,76.34,115790.0,-34136.0,73805.0,4.36,-22.77,63.74,26.46,129.0,-0.46,0.90,48.34,XR,NTT,BANKS
84,UJJIVANSFB,52.77,60.00,38.86,M-SC,44.09,120177.0,-22302.0,41822.0,3.44,-15.65,34.80,13.70,217.0,-0.53,0.94,41.84,X40N,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,4.33,M-LC,82.80,104768.0,26406.0,7627.0,2.99,33.70,7.28,43.43,71.0,3.46,0.82,40.13,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-59.76,L-SC,4.30,58464.0,-55085.0,95138.0,-4.87,-48.51,162.73,35.27,238.0,-0.58,0.46,43.09,XR,NTT,HOTELS
82,TRIDENT,37.35,48.00,58.66,M-SC,86.02,76274.0,-15980.0,42286.0,-2.86,-17.32,55.44,28.51,220.0,-0.38,0.59,30.52,XR,NTT,TEXTILES
70,SHALBY,261.39,327.00,836.20,M-SC,11.83,127387.0,-53756.0,99222.0,-1.96,-29.68,77.89,25.10,204.0,-0.54,0.99,1.96,XY24,NTT,HEALTHCARE
33,HEROMOTOCO,4311.81,6039.03,-3.33,H-MC,74.19,147875.0,-3038.0,63498.0,-1.96,-2.01,42.94,40.06,89.0,-0.05,1.15,20.95,AR,ATH,AUTO
7,ASIANTILES,75.41,137.00,6594.44,M-SC,98.92,73471.0,-20339.0,96960.0,-1.78,-21.68,131.97,81.67,203.0,-0.21,0.57,42.28,XR,NTT,CERAMICS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-13.50,X-LC,43.01,246863.0,16264.0,114865.0,-0.65,7.05,46.53,56.86,1.0,0.14,1.92,14.26,X40,BTT,IT
80,TCS,3841.06,4998.00,-17.44,X-LC,25.81,238229.0,-26804.0,106631.0,-0.31,-10.11,44.76,30.12,2.0,-0.25,1.86,6.84,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.11,X-LC,70.97,182880.0,-17430.0,76224.0,0.22,-8.70,41.68,29.35,4.0,-0.23,1.43,26.34,X200,ATH,IT
71,SIEMENS,4406.90,7969.85,-9.47,X-LC,72.04,81888.0,-28284.0,117362.0,0.30,-25.67,143.32,80.85,5.0,-0.24,0.64,20.61,AR,ATH,ELECTRICAL
32,HAVELLS,1588.02,2069.16,-8.59,X-LC,10.75,236231.0,-11500.0,86555.0,-0.83,-4.64,36.64,30.30,6.0,-0.13,1.84,8.03,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,11.94,H-SC,97.85,129502.0,31856.0,15825.0,0.31,32.62,12.22,48.82,138.0,2.01,1.01,43.86,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-0.31,M-LC,93.55,174859.0,34019.0,10474.0,4.77,24.15,5.99,31.59,76.0,3.25,1.36,51.89,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,82.80,104768.0,26406.0,7627.0,2.99,33.70,7.28,43.43,71.0,3.46,0.82,40.13,XY24,NTT,MISC
49,LICI,785.05,983.0,-2.96,H-LC,91.40,238762.0,44070.0,5014.0,0.87,22.64,2.10,25.21,29.0,8.79,1.86,32.03,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3576.0,-16.57,X-LC,36.02,92508.0,17386.0,14774.0,-0.76,23.14,15.97,42.81,11.0,1.18,0.72,16.02,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,38.95,M-SC,67.74,144134.0,6556.0,87850.0,2.57,4.77,60.95,68.62,182.0,0.07,1.12,24.95,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,11.94,H-SC,97.85,129502.0,31856.0,15825.0,0.31,32.62,12.22,48.82,138.0,2.01,1.01,43.86,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,8.99,H-SC,89.25,160726.0,22866.0,14899.0,0.33,16.59,9.27,27.39,118.0,1.53,1.25,34.48,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,6.01,H-MC,54.84,100734.0,15633.0,27470.0,-1.09,18.37,27.27,50.65,92.0,0.57,0.79,26.63,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,4.33,M-LC,82.80,104768.0,26406.0,7627.0,2.99,33.70,7.28,43.43,71.0,3.46,0.82,40.13,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,3.33,H-SC,51.61,116144.0,14140.0,30883.0,4.72,13.86,26.59,44.14,111.0,0.46,0.91,32.35,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-27.01,M-MC,55.91,142648.0,13496.0,73578.0,2.80,10.45,51.58,67.42,155.0,0.18,1.11,38.74,XR,NTT,BANKS
92,WIPRO,243.46,420.00,-10.62,M-LC,34.41,153438.0,2493.0,106962.0,-0.88,1.65,69.71,72.51,69.0,0.02,1.20,15.21,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,36.02,123352.0,16.0,136945.0,-0.22,0.01,111.02,111.05,108.0,0.00,0.96,20.86,AR,ATH,MISC
1,AARTIIND,487.04,919.00,36.22,M-SC,64.52,78290.0,-3046.0,75182.0,-0.10,-3.74,96.03,88.69,202.0,-0.04,0.61,32.02,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-22.76,H-SC,68.82,87600.0,-10380.0,32254.0,-0.85,-10.59,36.82,22.32,130.0,-0.32,0.68,27.05,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,82.80,104768.0,26406.0,7627.0,2.99,33.70,7.28,43.43,71.0,3.46,0.82,40.13,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-374.49,H-SC,100.00,116727.0,-16559.0,101156.0,-1.50,-12.42,86.66,63.47,131.0,-0.16,0.91,95.66,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,38.86,M-SC,44.09,120177.0,-22302.0,41822.0,3.44,-15.65,34.80,13.70,217.0,-0.53,0.94,41.84,X40N,NTT,BANKS
55,PFIZER,4245.50,6318.3,11.94,H-SC,97.85,129502.0,31856.0,15825.0,0.31,32.62,12.22,48.82,138.0,2.01,1.01,43.86,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-374.49,H-SC,100.00,116727.0,-16559.0,101156.0,-1.50,-12.42,86.66,63.47,131.0,-0.16,0.91,95.66,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,6594.44,M-SC,98.92,73471.0,-20339.0,96960.0,-1.78,-21.68,131.97,81.67,203.0,-0.21,0.57,42.28,XR,NTT,CERAMICS
55,PFIZER,4245.50,6318.30,11.94,H-SC,97.85,129502.0,31856.0,15825.0,0.31,32.62,12.22,48.82,138.0,2.01,1.01,43.86,X40,ATH,PHARMA
74,STARHEALTH,551.56,761.00,4.90,M-SC,96.77,171645.0,-28571.0,104600.0,-1.00,-14.27,60.94,37.97,194.0,-0.27,1.34,37.82,XY24,NTT,INSURANCE
77,TANLA,917.30,1963.11,-33.39,H-SC,95.70,163203.0,-76212.0,349173.0,0.93,-31.83,213.95,114.01,110.0,-0.22,1.27,49.68,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.88
1,25,41.25
2,50,70.05


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.15
XY25    14.42
X40     14.27
X40N    13.53
AR      12.44
XR      10.57
X200     1.43
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.58
M-SC    17.78
X-LC    17.31
H-MC     8.69
M-LC     7.90
H-LC     7.31
X-MC     5.65
M-MC     5.59
X-SC     2.80
L-SC     2.35
L-LC     1.15
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.46,-0.24,31.76
BANKS,9.97,-9.50,59.84
IT,9.90,-18.05,82.58
MISC,6.50,-29.90,87.36
FINANCE,5.95,-25.11,84.52
INSURANCE,5.44,4.51,24.11
PAINTS,5.38,-18.95,54.14
ELECTRICAL,4.97,-5.45,57.15
AUTO,4.41,-10.42,58.66


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2850297.0,26
AR,1251951.0,12
X40N,1210945.0,17
XR,1096421.0,14
X40,769001.0,11
XY25,449961.0,10
SR,196904.0,2
X200,76224.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2247206.0,21
M-SC,2033045.0,21
X-LC,798873.0,12
M-MC,549728.0,5
H-MC,511946.0,8
X-SC,348695.0,4
X-MC,308392.0,5
L-SC,297095.0,4
M-LC,292542.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      934233.0      6
M-SC       XY24      750135.0      6
H-SC       AR        544715.0      3
           X40N      499436.0      6
M-SC       X40N      467498.0      6
X-LC       X40       457467.0      5
M-MC       XY24      395509.0      3
M-SC       XR        293002.0      4
H-LC       AR        264816.0      3
H-SC       XR        238098.0      4
M-SC       X40       222070.0      2
H-MC       XY24      174909.0      2
M-LC       XY24      173658.0      4
L-SC       XR        157098.0      2
X-LC       AR        143808.0      2
X-MC       XY24      139285.0      2
X-SC       X40N      122079.0      2
X-LC       XY25      121374.0      4
M-SC       XY25      121139.0      1
X-SC       XR        114839.0      1
H-MC       XR        112844.0      1
X-SC       SR        111777.0      1
M-LC       XR        106962.0      1
X-MC       X40N       94462.0      2
M-SC       AR         94074.0      1
           SR         85127.0      1
M-MC       AR         80641.0      1
L-SC       XY24       79598.0      1
X-LC       X200       76224.0      1
X-MC       XY25       74645.0      1
H-MC       XY25       74485.0      1
M-MC       XR         73578.0      1
H-MC       AR         63498.0      1
L-SC       AR         60399.0      1
H-MC       X40        58740.0      2
L-LC       XY25       41382.0      1
H-SC       X40        30724.0      2
H-MC       X40N       27470.0      1
M-LC       XY25       11922.0      1
H-LC       XY25        5014.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
